# 実習の説明

## これは何か？

Jupyter notebook というRを使うためのインターフェースの一つです。

- Jupyter notebook の簡単な説明（日本語） https://datumstudio.jp/blog/795

特に、今回は JupyterHub という形で（みなさんの手元のPCではなく）オンラインでJupyter notebookを使う環境を提供しています。
これは、講義などで同一の環境でできるのに便利です。


## 何をするか？

非小細胞肺癌 (non-small-cell lung cancer; NSCLC) において、腫瘍組織内に存在する間質細胞から腫瘍細胞にシグナルのクロストークが送って腫瘍が活性化される可能性を探ります。データは、NSCLCモデルマウスと野生型のマウスの肺から、セルソーターで分けたマクロファージ、単球細胞、好中球、上皮細胞のRNA-seqデータです。

> このようなRNA-seqデータを、「変数が全遺伝子、条件が細胞型xマウス（野生型 or 腫瘍）、値が発現量である表」として想像できると、あとの解析がスムーズです

元ネタはこちら: 

- Toi _et al_., Transcriptome Analysis of Individual Stromal Cell Populations Identifies Stroma-Tumor Crosstalk in Mouse Lung Cancer Model, Cell Reports (2015) http://dx.doi.org/10.1016/j.celrep.2015.01.040 

ソフトウェアやアノテーションデータが著者らのウェブページで公開されています。また、NGSデータの生データや処理済みデータはGene Expression Omnibus (GEO)より公開されています。

- ソフトウェアやアノテーションデータ http://209.160.41.231/u54/CCCExplorer/
- Gene Expression Omnibus (GEO)のNGSデータのページ https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE59831

## どうやるか？

腫瘍組織内間質細胞のリガンドから腫瘍細胞の受容体へのシグナルのクロストークがあるとしたら、

- 腫瘍モデルマウスでは野生型に比べ、リガンドが発現上昇しているだろう
- 腫瘍モデルマウスの上皮組織では、そのリガンドの受容体が発現しているだろう

そのようなリガンドと受容体のペアを見つけるのに必要なデータは？

- 既知のリガンド-受容体ペアのデータベース（ヒト）
- ヒト-マウスオーソログの対応表
- 腫瘍組織内間質細胞で発現上昇がみられた遺伝子のリスト
- 腫瘍細胞での各遺伝子の発現量

それらのデータを組み合わせればよさそう

![image.png](https://ars.els-cdn.com/content/image/1-s2.0-S2211124715000650-fx1.jpg)
(Image from http://dx.doi.org/10.1016/j.celrep.2015.01.040)

# 練習

## A01: 必要なパッケージを読み込む
パッケージ（ライブラリ）を使えるようにするためにロードする

In [ ]:
library(tidyverse)
library(magrittr)

## A02: プロットを大きくすぎない呪文

In [ ]:
# プロットを大きくしすぎない呪文
options(repr.plot.width=4, repr.plot.height=4)

## A03: 発現変動遺伝子のデータを読み込む

ここではマクロファージのデータ (`data/gene_exp.diff`) を例に、データのいじくりかたをみていきます。

In [ ]:
# データの読み込み
deg_macrophage = read_tsv("data/gene_exp.diff")

In [ ]:
# とりあえず何行何列かみてみる
dim(deg_macrophage)

In [ ]:
# データの頭の部分だけ確認
head(deg_macrophage)

## A04: 発現変動遺伝子のデータを加工する

ふむふむ

- 縦に遺伝子名のようなものが並んでいる。各行が遺伝子、各列が変数になっている
- `p_value`や`q_value`は発現変動を判定する統計検定の結果だろう
- `log2(fold_change)` は `value_2`/`value_1`をlog2変換したものだろう
    - ちなみに、`value_1` は正常組織でのマクロファージ、 `value_2` は腫瘍組織内マクロファージ (intratumoral macropahges)でのFPKMの平均値
    - これは __メタデータ__ をみないとわからない

In [ ]:
# 使いたい列だけにする
deg_macrophage %<>% select(gene_id, value_1, value_2, `log2(fold_change)`, p_value, q_value)

In [ ]:
# データの形が変わったことを確認
head(deg_macrophage)

In [ ]:
# p-valueの分布を確認する
hist(deg_macrophage$p_value)

In [ ]:
# q-valueの分布を確認する
hist(deg_macrophage$q_value)

In [ ]:
# `log2(fold_change)`の分布を確認する
hist(deg_macrophage$`log2(fold_change)`)

In [ ]:
# log2FC と q_valueの関係を眺めてみる (volcano plot)
plot(deg_macrophage$`log2(fold_change)`, -log10(deg_macrophage$q_value))

## A05: 発現変動遺伝子の数を数える

発現変動遺伝子の数を数えましょう


In [ ]:
# 腫瘍組織内マクロファージで多い (FC > 1.5, q_value < 0.1) 遺伝子の数は？
deg_macrophage %>% filter(`log2(fold_change)` > log2(1.5), q_value < 0.1) %>% nrow

この中で、既知の発現変動遺伝子（条件間で変動が観察されることがすでに知られている遺伝子）が含まれているかを調べてみましょう

- "intratumoral macrophages express increased cathepsin K, COX-2, MMP-9, PDGF-B, uPA, VEGFA, and HGF (Wang et al., 2011)"
    - Wang et al., 2011 http://www.sciencedirect.com/science/article/pii/S0169500211002546

In [ ]:
# ポジコンがとれるかの確認 (生物学的なQC)
deg_macrophage %>% filter(`log2(fold_change)` > log2(1.5), q_value < 0.1) %>% filter(gene_id == "Vegfa")
deg_macrophage %>% filter(`log2(fold_change)` > log2(1.5), q_value < 0.1) %>% filter(gene_id == "Mmp9")

In [ ]:
# 正常組織内マクロファージで多い (FC > 1/1.5, q_value < 0.1) の遺伝子の数は？
deg_macrophage %>% filter(`log2(fold_change)` > -log2(1.5), q_value < 0.1) %>% nrow

------

------

------


# 本番

## A06: 腫瘍組織内間質細胞で発現上昇する遺伝子のリストを読み取る

In [ ]:
# 例として野生型と腫瘍のマクロファージで発現変動検定をした結果を読み込みます
deg_macrophage = read_tsv("data/gene_exp.diff")

# 行数と列数
dim(deg_macrophage)

## A07: 腫瘍組織内間質細胞で発現上昇する遺伝子のリストを加工する

In [ ]:
# 使いたい列だけにする
deg_macrophage %<>% select(gene_id, value_1, value_2, `log2(fold_change)`, p_value, q_value)

# データの中身の確認
deg_macrophage %>% head

In [ ]:
# とある条件で腫瘍組織内間質細胞で発現上昇する遺伝子をフィルタリング
deg_macrophage_tumor = deg_macrophage %>% filter(value_2 > 2 ,`log2(fold_change)` > log2(1.5), q_value < 0.1)

# 何行 (何遺伝子)
deg_macrophage_tumor %>% nrow

## A08: 腫瘍細胞で発現する遺伝子のリストを読み取る


In [ ]:
# 遺伝子発現量 (FPKM) をまとめた表を読み込む (Cufflinksの出力だとおもう)
fpkm_epi = read_tsv("data/genes.read_group_tracking")

# 何行何列か
dim(fpkm_epi)

# データの最初の数行をみる
fpkm_epi %>% head

## A09: 腫瘍細胞で発現する遺伝子のリストを加工する

In [ ]:
# 必要な列だけにする
fpkm_epi %<>% select(tracking_id, FPKM)

# 確認
fpkm_epi %>% head

In [ ]:
# とある条件で腫瘍細胞で発現する遺伝子をフィルタリング
fpkm_epi_tumor = fpkm_epi %>% filter(FPKM > 2)

# 何行 (何遺伝子)
fpkm_epi_tumor %>% nrow

## A10: ヒト-マウスのオーソログ関係の表を読み取る

`data/HOM_MouseHumanSequence.rpt` は Mouse Genome Informatics database から取得されたヒト-マウスオーソログリスト

In [ ]:
# データの読み込み
dfhom = read_tsv("data/HOM_MouseHumanSequence.rpt")

# 何行何列
dim(dfhom)

# データの頭をみる
head(dfhom)

## A11: ヒト-マウスのオーソログ関係の表を加工する

In [ ]:
# 関係ありそうなところだけにする
dfhom %<>% select(`HomoloGene ID`, Symbol, `Common Organism Name`)

# 確認
dfhom %>% head

実は上の表は 1対1オーソログだけでなく、1対多オーソログも含まれていたので、ちょっと面倒です。
このような場合、1対1オーソログのみに絞ることもありますし、全部残す場合もあります。ケースバイケース。

いずれにせよ、上の形では扱いづらいので、ヒトとマウスのオーソログの対応が1行ごとに並んだ形にしましょう。

In [ ]:
# マウスの行とヒトの表をそれぞれ抜き出し、 `full_join` で joinしている
dfhom2 = full_join(
    dfhom %>%
        filter(`Common Organism Name` == "mouse, laboratory") %>%
        select(-`Common Organism Name`),
    dfhom %>% 
        filter(`Common Organism Name` == "human") %>%
        select(-`Common Organism Name`),
    by = "HomoloGene ID"
)

# 列名を変更する
dfhom2 %<>% dplyr::rename(mouse=Symbol.x, human=Symbol.y)

# 何行何列
dfhom2 %>% dim

# データの頭
dfhom2 %>% head

## A12: リガンド-受容体関係の表を読み込む

（実は`From` がリガンドで、 `To` が受容体）

In [ ]:
# 読み込み
dflr = read_tsv("data/LR_manual_revised.txt")

# 何行何列
dflr %>% dim

# 頭
dflr %>% head

## A13: 遺伝子のリストとヒト-マウスオーソログの表を結合する

遺伝子のリストとヒト-マウスオーソログの表を結合することで、マウスの遺伝子リストであってもヒトの遺伝子に関するデータベースの情報と照合することができるようになります。

そのために、JOIN をします。JOINは２つの表x, yそれぞれの列を比較し、x,yを結合した新しい表を作る操作です。

比較する列をキーと呼びます。表 x, y のキーがユニークで同一であれば単に横に連結すればいいですが、実際には、xもしくはyまたは両方で足りない/余分なキーがある場合があり、そのような場合にどう処理するかによって、JOINに様々な種類があります。

- join について https://qiita.com/matsuou1/items/b1bd9778610e3a586e71

In [ ]:
dfhom2 %>% colnames

In [ ]:
# ヒト-マウスオーソログ関係の表と遺伝子リスト表を結合する
## `left_join` は一番目の表（データフレーム) の行は全部残して、JOINを行う
deg_macrophage_tumor_human = left_join(deg_macrophage_tumor, dfhom2, by=c("gene_id" = "mouse"))

# head
## deg_macrophage_tumor にあっても dfhom2 になかった列では、 `HomoloGene ID`やhumanが欠損値 (NA) になっている
deg_macrophage_tumor_human %>% head

# human が NA でないの行だけにする
deg_macrophage_tumor_human　%<>% filter(!is.na(human))

deg_macrophage_tumor_human %>% head

In [ ]:
# ヒト-マウスオーソログ関係の表と遺伝子リスト表を結合する
fpkm_epi_tumor_human = left_join(fpkm_epi_tumor, dfhom2, by=c("tracking_id" = "mouse"))

# head
fpkm_epi_tumor_human %>% head

# human が NA でないの行だけにする
fpkm_epi_tumor_human　%<>% filter(!is.na(human))

# 何行何列
fpkm_epi_tumor_human %>% dim

fpkm_epi_tumor_human　%>% head

## A14: リガンド-受容体関係の情報に発現/発現変動遺伝子の情報を加える

`A %in% B` はAの各要素がBに含まれるかをTRUE/FALSEのベクトルで返す関数です。

In [ ]:
# ligand_up_in_tumor_macrophage という列を追加する。deg_macrophage_tumor_human に リガンドが含まれるならばTRUE、そうでないならばFALSSEとなる
## 
dflr %>% 
    mutate(ligand_up_in_tumor_macrophage = From %in% deg_macrophage_tumor_human$human) -> dflr

# head
dflr %>% head

# ligand_up_in_tumor_macrophage の数を集計
dflr %>% group_by(ligand_up_in_tumor_macrophage) %>% summarise(n = n())

In [ ]:
# receptor_expressed_in_tumor_macrophage という列を追加する。fpkm_epi_tumor_human に 受容体が含まれるならばTRUE、そうでないならばFALSSEとなる
## 
dflr %<>% mutate(receptor_expressed_in_tumor_macrophage = To %in% fpkm_epi_tumor_human$human)

# head
dflr %>% head

# receptor_expressed_in_tumor_macrophage の数を集計
dflr %>% group_by(receptor_expressed_in_tumor_macrophage) %>% summarise(n = n())

In [ ]:
# `ligand_up_in_tumor_macrophage` と  `receptor_expressed_in_tumor_macrophage` が両方TRUEである行を探す
dflr_pair = dflr %>% filter(ligand_up_in_tumor_macrophage == TRUE, receptor_expressed_in_tumor_macrophage == TRUE)

# dim
dflr_pair %>% dim

# head
dflr_pair %>% head

先行研究で、腫瘍組織内マクロファージで発現上昇が見られるリガンドも入っていた

- "intratumoral macrophages express increased cathepsin K, COX-2, MMP-9, PDGF-B, uPA, VEGFA, and HGF (Wang et al., 2011)"
    - Wang et al., 2011 http://www.sciencedirect.com/science/article/pii/S0169500211002546

In [ ]:
dflr_pair %>% filter(From == "HGF")

## A15: とりたいものがとれてきているか可視化して確認する

GEOのFPKMの発現量行列を用いて、上の方法でとってきたリガンドレセプターのペアが本当に

- 腫瘍組織内マクロファージで多いリガンドか？
- 腫瘍細胞で発現している受容体か？

ということを確かめました。

GEOのデータはあらかじめダウンロードしておきました。

GEOのページのメタデータをみないとわからないのですが、Tum1, Tum2, Tum3 が腫瘍組織内マクロファージ、WT1, WT2が野生型組織でのマクロファージ、Tum9, Tum10, Tum11が腫瘍細胞です。

In [ ]:
# データの読み込み
df_fpkm = read_tsv("data/GSE59831_processed_data_FPKM.txt")

# dim
df_fpkm %>% dim

# 様子を見る
df_fpkm %>% head

# 列名だけを表示
df_fpkm %>% colnames

In [ ]:
# リガンドだけを可視化
## うまくいっているなら、Tum1, Tum2, Tum3 (腫瘍組織内マクロファージ)で多く、WT1, WT2 (野生型) で少ない

# dflr_pair に含まれるリガンドだけに絞り、 as.matrix　で行列にする
x = df_fpkm %>% 
        select(human_gene_symbol, Tum1, Tum2, Tum3, WT1, WT2, Tum9, Tum10, Tum11) %>%
        filter(human_gene_symbol %in% dflr_pair$From) %>% 
        select(-human_gene_symbol) %>% 
        as.matrix()

# 何行何列
dim(x)

# ヒートマップをプロット
heatmap(x, Colv = FALSE)

In [ ]:
# 受容体だけを可視化
## うまくいっているなら、Tum9, Tum10, Tum11 で多い

# dflr_pair に含まれるリガンドだけに絞り、 as.matrix　で行列にする
x = df_fpkm %>% 
        select(human_gene_symbol, Tum1, Tum2, Tum3, WT1, WT2, Tum9, Tum10, Tum11) %>%
        filter(human_gene_symbol %in% dflr_pair$To) %>% 
        select(-human_gene_symbol) %>% 
        as.matrix()

# 何行何列
dim(x)

# ヒートマップをプロット
heatmap(x, Colv = FALSE)


----

----

----


## このあとどうするか？

今のままでは、候補がかなり多いため、仮説を検証する実験をするのは難しそうです。また、この予測がどのくらい正しいのかも不明です。

元論文では、このリガンド-受容体ペアを足がかりに、以下のような解析をします。

- 受容体の下流のシグナル伝達経路から転写因子があるか探す（パスウェイデータベースを参照する）
- それらの下流の転写因子がターゲットとする遺伝子を探す（パスウェイデータベースや遺伝子制御関係のデータベースを参照する）
- それらのターゲット遺伝子群にランダムに比べて発現変動遺伝子が濃縮しているかを調べる（統計検定）

このような解析を加えることで、単にリガンドと受容体が共起だけでなく、さらに信頼性のあるペアを探すことができます。

ちなみに、著者らはこの方法をCCCExplorerと名付けています。詳しくは元の論文をご覧ください:

- Toi _et al_., Transcriptome Analysis of Individual Stromal Cell Populations Identifies Stroma-Tumor Crosstalk in Mouse Lung Cancer Model, Cell Reports (2015) http://dx.doi.org/10.1016/j.celrep.2015.01.040 


## 小まとめ

このようにIDによる統合を繰り返すだけでもいろいろな仮説を立てられます。また、NGS解析とは言いつつ、NGSデータ以外の生命科学データを合わせることで、単なる発現変動遺伝子のリストよりも深い知見が得られるうることがみえてきたかと思います。
